In [52]:
from sigmaepsilon.solid.fem import FemMesh, TET4, Structure
from neumann.linalg import normalize
import pyvista as pv
import numpy as np
import tetgen

In [53]:
# geometry [cm]
d = 10.0
h = 1.5
a = 1.5
b = 0.5

# cylinder mesh resolution
res = [10, 100, 30]

# mass of the ball [kg]
m = 10
g = 9.81

# material
E = 1200
nu = 0.2

In [54]:
cyl = pv.CylinderStructured(center=(0.0, 0.0, h/2), direction=(0.0, 0.0, 1.0),
                            radius=np.linspace(b/2, a/2, res[0]), height=h, 
                            theta_resolution=res[1], z_resolution=res[2])
cyl_surf = cyl.extract_surface().triangulate()
tet = tetgen.TetGen(cyl_surf)
tet.tetrahedralize(order=1, mindihedral=10, minratio=1.1, quality=True)
grid = tet.grid
coords = np.array(grid.points).astype(float)
topo = grid.cells_dict[10].astype(np.int32)

In [55]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 2]) <= 1e-12
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 3), dtype=bool)
fixity[ebcinds, :] = True

In [56]:
# Hooke model
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E /(1-nu**2))

In [57]:
# natural boundary conditions
cond_nbc_1 = coords[:, 2] >= (h - 1e-8)
cond_nbc_2 = np.abs((coords[:, 0]**2 + coords[:, 1]**2)**(1/2) - b/2) <= 0.01
cond_nbc = cond_nbc_1 & cond_nbc_2
nbcinds = np.where(cond_nbc)[0]
q = (m * g * d) / (len(nbcinds) * np.sqrt(d**2-b**2))
c = np.array([0, 0, h + np.sqrt(d**2-b**2)/2])
foo = lambda x : normalize(x - c) * q
loads = np.zeros((coords.shape[0], 3))
loads[nbcinds] = [foo(coords[nID]) for nID in nbcinds]

In [58]:
mesh = FemMesh(coords=coords, topo=topo, celltype=TET4, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [64]:
structure.plot()

In [66]:
structure.linsolve()
structure.summary

{'preproc': {'sparsify': False},
 'proc': {'time [ms]': 40.96508026123047,
  'N': 93681,
  'use_umfpack': True,
  'permc_spec': 'COLAMD',
  'solver': 'pardiso'},
 'postproc': {},
 'number of elements': 143574,
 'number of nodes': 31227,
 'dofs per node': 3}

In [61]:
dofsol = structure.mesh.pointdata.dofsol.to_numpy()
dofsol[:, 2].min()

-0.5627807477044733

In [62]:
structure.volume(), np.pi * (a**2 - b**2) * h / 4

(2.3546130172499566, 2.356194490192345)

In [63]:
mb = structure.to_pv()